In [10]:
from knuckledragger.kernel import *
from z3 import *
class Datatype(z3.Datatype):
    def create(self):
        DT = super().create()  # z3 already checks for positivity.
        PredSort = ArraySort(DT, BoolSort())
        # construct an induction principle.
        P = FreshConst(PredSort, prefix="P")
        hyps = []
        for i in range(DT.num_constructors()):
            constructor = DT.constructor(i)
            args = [
                FreshConst(constructor.domain(j), prefix="a")
                for j in range(constructor.arity())
            ]
            acc = P[constructor(*args)]
            for arg in args:
                if arg.sort() == DT:
                    acc = QForAll([arg], P[arg], acc)
                else:
                    acc = ForAll([arg], acc)
            hyps.append(acc)
        x = FreshConst(DT, prefix="x")
        conc = ForAll([x], P[x])
        induct = Lambda([P], Implies(And(hyps), conc))
        induct_ax = trust(ForAll([P], induct[P] == True))

        # recursor

        # store inside the data type?
        DT.induct = induct
        DT.induct_ax = induct_ax
        #DT.rec = rec
        return DT

def recursor(name, *cases, DT):
    assert all(case.range() == DT for case in cases)
    f = z3.RecFunction(name, DT, )  # also freshness needs to be checked


def define_rec(name, args, body, measure=None):
    sig = [arg.sort() for arg in args]
    f = z3.RecFunction(name, *sig)  # also freshness needs to be checked. Yikes.
    # TODO check termination and totality of definition
    RecAddDefinition(f, body)
    fun_def = infer([], ForAll(args, f(*args) == body))
    return f, fun_def


Wrappers for algebraic datatypes deriving induction principles, recursion principles, others? Some kind of well foundedness? What does isabelle define?

In [23]:

List = Datatype("List")
List.declare("cons", ("car", IntSort()), ("cdr", List))
List.declare("nil")
List = List.create()
List.induct_ax

List2 = Datatype("List")
List2.declare("nil")
List2 = List2.create()
List2.induct_ax

List2 == List # ruh roh

List.induct_ax
List2.induct_ax
prove(List2.nil == List.nil) # yikes
List.num_constructors()
List2.num_constructors()
# ok so we overwrite the old list.

# This possibly invlaidates any oldtheorem

proved


1

I guess one kind of insane option is to allow delayed validation.

We might need to rexport everything from z3 with some stuff wrapped. That's a somewhat nuclear options.
Or cache when it comes into trust?
An even more nuclear option is make my own adt

